In [1]:
# Asset Management System - Colab Compatible
import sqlite3
import pandas as pd

# Connect to SQLite database (in-memory for demo, use 'assets.db' for file storage)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE assets (
    asset_id INTEGER PRIMARY KEY AUTOINCREMENT,
    asset_name TEXT NOT NULL,
    category TEXT NOT NULL,
    value REAL NOT NULL,
    status TEXT NOT NULL
)
""")

# ---------- CRUD Functions ----------

def add_asset(asset_name, category, value, status="Active"):
    cursor.execute("INSERT INTO assets (asset_name, category, value, status) VALUES (?, ?, ?, ?)",
                   (asset_name, category, value, status))
    conn.commit()
    print(f"✅ Asset '{asset_name}' added successfully!")

def update_asset(asset_id, asset_name=None, category=None, value=None, status=None):
    updates = []
    params = []
    if asset_name: updates.append("asset_name=?"); params.append(asset_name)
    if category: updates.append("category=?"); params.append(category)
    if value: updates.append("value=?"); params.append(value)
    if status: updates.append("status=?"); params.append(status)
    params.append(asset_id)
    query = "UPDATE assets SET " + ", ".join(updates) + " WHERE asset_id=?"
    cursor.execute(query, tuple(params))
    conn.commit()
    print(f"🔄 Asset ID {asset_id} updated successfully!")

def delete_asset(asset_id):
    cursor.execute("DELETE FROM assets WHERE asset_id=?", (asset_id,))
    conn.commit()
    print(f"🗑️ Asset ID {asset_id} deleted successfully!")

def view_assets():
    df = pd.read_sql("SELECT * FROM assets", conn)
    if df.empty:
        print("No assets found.")
    else:
        display(df)

# ---------- Sample Analytics ----------
def asset_summary():
    df = pd.read_sql("SELECT * FROM assets", conn)
    if df.empty:
        print("No assets to analyze.")
        return
    print("📊 Asset Summary Report:")
    summary = df.groupby("category")["value"].sum().reset_index()
    display(summary)

# ---------- Demo ----------
add_asset("Laptop Dell XPS", "Electronics", 1200)
add_asset("Office Chair", "Furniture", 250)
add_asset("AWS Cloud Subscription", "Software", 600)
add_asset("MacBook Pro", "Electronics", 2000, status="In Repair")

print("\n📋 Current Assets:")
view_assets()

print("\n🔄 Updating Asset 2 (Chair value changed)...")
update_asset(2, value=300)

print("\n📋 Assets after update:")
view_assets()

print("\n🗑️ Deleting Asset 3 (AWS Cloud Subscription)...")
delete_asset(3)

print("\n📋 Assets after deletion:")
view_assets()

print("\n📊 Generating Asset Summary...")
asset_summary()


✅ Asset 'Laptop Dell XPS' added successfully!
✅ Asset 'Office Chair' added successfully!
✅ Asset 'AWS Cloud Subscription' added successfully!
✅ Asset 'MacBook Pro' added successfully!

📋 Current Assets:


,asset_id,asset_name,category,value,status
0,1,Laptop Dell XPS,Electronics,1200.0,Active
1,2,Office Chair,Furniture,250.0,Active
2,3,AWS Cloud Subscription,Software,600.0,Active
3,4,MacBook Pro,Electronics,2000.0,In Repair



🔄 Updating Asset 2 (Chair value changed)...
🔄 Asset ID 2 updated successfully!

📋 Assets after update:


,asset_id,asset_name,category,value,status
0,1,Laptop Dell XPS,Electronics,1200.0,Active
1,2,Office Chair,Furniture,300.0,Active
2,3,AWS Cloud Subscription,Software,600.0,Active
3,4,MacBook Pro,Electronics,2000.0,In Repair



🗑️ Deleting Asset 3 (AWS Cloud Subscription)...
🗑️ Asset ID 3 deleted successfully!

📋 Assets after deletion:


,asset_id,asset_name,category,value,status
0,1,Laptop Dell XPS,Electronics,1200.0,Active
1,2,Office Chair,Furniture,300.0,Active
2,4,MacBook Pro,Electronics,2000.0,In Repair



📊 Generating Asset Summary...
📊 Asset Summary Report:


,category,value
0,Electronics,3200.0
1,Furniture,300.0
